# openEO Demonstration

## Setup

In [ ]:
import os
import openeo
import json
import xarray
import matplotlib.pyplot as pyplot

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

Load `eoepca state` environment

In [ ]:
load_eoepca_state()

In [ ]:
platform_domain = os.environ.get("INGRESS_HOST")

openeo_backend = f"openeo.{platform_domain}"
authentication_method = "device-code-flow" # or authorization-code

username = os.environ.get("KEYCLOAK_TEST_USER")
password = os.environ.get("KEYCLOAK_TEST_PASSWORD")
client_id = os.environ.get("OPENEO_CLIENT_ID")

collection_id = "TestCollection-LonLat16x16"
temporal_extent = "2024-09"
spatial_extent = {"west": 3, "south": 51, "east": 5, "north": 53}

log_output_file = "openeo_log.json"

In [ ]:
print(f"Parameters:\n"
f"Platform domain: {platform_domain}\n"
f"OpenEO backend: {openeo_backend}\n"
f"Authentication method: {authentication_method}\n"
f"Collection ID: {collection_id}\n"
f"Temporal extent: {temporal_extent}\n"
f"Spatial extent: {spatial_extent}")

## Connect to the openEO backend

In [ ]:
connection = openeo.connect(url=openeo_backend)
if authentication_method == "device-code-flow":
    connection.authenticate_oidc()
    print("Visit this backend through Web Editor:\n" + connection.web_editor(anonymous=True))
elif authentication_method == "authorization-code":
    access_token = get_access_token(username, password, client_id)
    auth_token = f"oidc/eoepca/{access_token}"
    connection.session.headers.update({"Authorization": f"Bearer {auth_token}"})

## Exploration

Available collections:

In [ ]:
# collection_exists
available_collections = connection.list_collection_ids()
assert collection_id in available_collections

Available processes:

In [ ]:
# list_processes
[p["id"] for p in connection.list_processes()[:10]]

## Simple processing

In [ ]:
# process_execution
process_graph = {
    "add": {
        "process_id": "add",
        "arguments": {"x": 3, "y": 5},
        "result": True,
    },
}

result = connection.execute(process_graph)
result

## Raster processing

Raster processing with built-in dummy collection (which does not require direct access to actual EO data).

In [ ]:
# data_loading
cube = connection.load_collection(
    collection_id=collection_id,
    temporal_extent=temporal_extent,
    spatial_extent=spatial_extent,
    bands=["Longitude", "Latitude", "Day"],
)

In [ ]:
%%time
cube.download("openeo-lonlat1616.nc")

Inspect downloaded results:

In [ ]:
# raster_download
ds = xarray.load_dataset("openeo-lonlat1616.nc")
ds

In [ ]:
fig, axes = pyplot.subplots(ncols=2, figsize=(7, 3), dpi=72, sharey=True)

ds["Longitude"].isel(t=0).plot.imshow(ax=axes[0])
ds["Latitude"].isel(t=0).plot.imshow(ax=axes[1]);

## Real raster processing with `load_stac`

`load_stac` is an standard openEO process that allows to load EO data that is hosted externally through the STAC standard.

In [ ]:
# raster_open
stac_url = "https://raw.githubusercontent.com/Open-EO/openeo-geopyspark-driver/refs/heads/master/docker/local_batch_job/example_stac_catalog/collection.json"

cube = connection.load_stac(
    url=stac_url,
    temporal_extent=["2023-06-01", "2023-06-09"],
    spatial_extent={"west": 5.07, "south": 51.215, "east": 5.08, "north": 51.22},
    bands=["B04", "B03", "B02"],
)
cube

Simple "band math" based processing, to calculate some NDVI-like index:

In [ ]:
# band_math
b04 = cube.band("B04")
b02 = cube.band("B02")
n42 = (b04 - b02) / (b04 + b02)

Trigger processing and download result synchronously:

In [ ]:
# nc_download
n42.download("openeo-load_stac-n42-sync.nc")

Inspect result:

In [ ]:
# xarray_load_dataset
ds = xarray.load_dataset("openeo-load_stac-n42-sync.nc")
ds

In [ ]:
# plot_figure
fig, ax = pyplot.subplots(dpi=75, figsize=(5, 5))

ds["var"].isel(t=0).plot.imshow(ax=ax, cmap="viridis");

In [ ]:
if test_results:
    for test, result in test_results.items():
        print(f"{test}: {result['status']} - {result['message']}")
    json.dump(test_results, open(log_output_file, "w"), indent=2)